# Poses and transforms (homogeneous coordinates)
## Camera sensor

This notebook follows the example of passive rotations to create poses as in [From_pose_to_transform](./From_pose_to_transform.ipynb), but with the addition of a transform that rotates the sensor coordinate system so it matches with the usual OpenCV convention (x-right, y-down, z-forward in the optical axis).

In [ ]:
%matplotlib notebook

In [ ]:
from PIL import Image
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
from vcd import utils


def plot_cs(pose_wrt_ref, name, L=1):
    # Explore the coordinate systems defined for this scene
    axis = np.array([[0, L, 0, 0, 0, 0],
            [0, 0, 0, L, 0, 0],
            [0, 0, 0, 0, 0, L],
            [1, 1, 1, 1, 1, 1]])  # matrix with several 4x1 points
    pose_wrt_ref = np.array(pose_wrt_ref).reshape(4, 4)
    axis_ref = pose_wrt_ref.dot(axis)
    origin = axis_ref[:, 0]
    x_axis_end = axis_ref[:, 1]
    y_axis_end = axis_ref[:, 3]
    z_axis_end = axis_ref[:, 5]
    ax.plot([origin[0], x_axis_end[0]], [origin[1], x_axis_end[1]], [origin[2], x_axis_end[2]], 'r-')
    ax.plot([origin[0], y_axis_end[0]], [origin[1], y_axis_end[1]], [origin[2], y_axis_end[2]], 'g-')
    ax.plot([origin[0], z_axis_end[0]], [origin[1], z_axis_end[1]], [origin[2], z_axis_end[2]], 'b-')

    ax.text(origin[0], origin[1], origin[2], r'{}'.format(name))
    ax.text(x_axis_end[0], x_axis_end[1], x_axis_end[2], 'X')
    ax.text(y_axis_end[0], y_axis_end[1], y_axis_end[2], 'Y')
    ax.text(z_axis_end[0], z_axis_end[1], z_axis_end[2], 'Z')

# Let's deefine the rotation and position of the Sensor Coordinate System SCS (e.g. FV: frontal view)
# with respect to the Local Coordinate System LCS (aka Vehicle COordinate System)
rz = (5 * np.pi) / 180.0  # yaw
ry = (15 * np.pi) / 180.0  # pitch
rx = (0 * np.pi) / 180.0  # roll
R_fv_wrt_lcs = utils.euler2R([rz, ry, rx], seq=utils.EulerSeq.ZYX)

C_fv_wrt_lcs = np.array([[2.5], [0.0], [0.5]])
P_fv_wrt_lcs = utils.create_pose(R_fv_wrt_lcs, C_fv_wrt_lcs)

T_fv_to_lcs = P_fv_wrt_lcs  # So X_lcs = T_fv_to_lcs * X_fv
T_lcs_to_fv = utils.inv(T_fv_to_lcs)  # So X_fv = T_lcs_to_fv * X_lcs (to project a 3D point to the sensor coordinate system)

# Visualize
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Let's plot the Vehicle coordinate system
P_lcs_wrt_lcs = np.array([[1.0, 0.0, 0.0, 0.0],[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]]) 
plot_cs(P_lcs_wrt_lcs, 'lcs', L=2)
plot_cs(P_fv_wrt_lcs, 'fv', L=1)

ax.set_xlim(-3, 3)
ax.set_ylim(-3, 3)
ax.set_zlim(0, 6)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')


To match the OpenCV convention (the camera coordinate system has x-right, y-down), an additional transformation needs to be applied

In [ ]:
R_opcv_wrt_fv = utils.euler2R([-np.pi/2.0, 0, -np.pi/2.0], seq=utils.EulerSeq.ZYX)

# This is post-multiplied, so the rz, ry are applied conceptually against the original LCS system
# just like in the example above, and only when the FV is in place, then rotate to match OpenCV system
R_opcv_wrt_lcs = R_fv_wrt_lcs @ R_opcv_wrt_fv

# If R_fv_wrt_lcs = R_opcv_wrt_fv @ R_fv_wrt_lcs, then rz, ry, rx are conceptually applied against the OpenCV-related coordinates
# IT DEPENDS WHAT THE provided (rz, ry, rx) refer to!!

P_opcv_wrt_lcs = utils.create_pose(R_opcv_wrt_lcs, C_fv_wrt_lcs)

T_opcv_to_lcs = P_opcv_wrt_lcs  # So X_lcs = T_fv_to_lcs * X_fv
T_lcs_to_opcv = utils.inv(T_opcv_to_lcs)  # So X_opcv = T_lcs_to_opcv * X_lcs (to project a 3D point to the sensor coordinate system matching OpenCV criteria)


# Visualize
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Let's plot the Vehicle coordinate system
P_lcs_wrt_lcs = np.array([[1.0, 0.0, 0.0, 0.0],[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]]) 
plot_cs(P_lcs_wrt_lcs, 'lcs', L=2)
plot_cs(P_opcv_wrt_lcs, 'opcv', L=1)

ax.set_xlim(-2, 2)
ax.set_ylim(-2, 2)
ax.set_zlim(0, 4)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

### Projection into image
Having the camera coordinate system matching the OpenCV convention simplifies the subsequent projection process from 3D points to 2D space in the image plane.

From projective geometry literature we can see the projection of a 3D point $\mathbf{X}$ expressed in the camera coordinate system (already following x-right, y-down, z-forward convention) into $(u,v)$ pixel positions in the image is:

$$
(u,v,1)^\top = K \mathbf{X}
$$

Usually K is a $3\times 3$ matrix as follows:

$$
K=\begin{pmatrix}f_x & 0 & c_x\\ 0 & f_y & c_y\\ 0 & 0 & 1\end{pmatrix}
$$

Where $f_x$ and $f_y$ is the focal length at each axis, and $c_x$ and $c_y$ is the principal point of the camera.
NOTE: If we haven't converted the camera coordinate system to match this convention, the matrix $K$ would have negative focal lengths and swapped y-z axis which causes confusion and problems with projection SW packages.

Let's create a camera calibration matrix and project points:

In [ ]:
img_width_px = 800
img_height_px = 600
f_x = 1000
f_y = 1000
c_x = img_width_px/2
c_y = img_height_px/2

In [ ]:
K = np.array([[f_x, 0, c_x, 0], [0, f_y, c_y, 0], [0, 0, 1, 1]])

Now let's create some points in LCS, converto to camera coordinate systems, and project

In [ ]:
step_x = 0.1
step_y = 0.1
x_min = 5
x_max = 10
y_min = -3
y_max = 3
num_points_x = int((x_max - x_min)/step_x)
num_points_y = int((y_max - y_min)/step_y)
xm, ym, zm = utils.generate_grid(x_params=(x_min, x_max, num_points_x + 1), y_params=(y_min, y_max, num_points_y+1), z_params=(0, 0, 1))
points3d_4xN = utils.grid_as_4xN_points3d(xm, ym, zm)

#Transform from LCS to FV:
T_lcs_to_fv = utils.inv(P_fv_wrt_lcs)
points3d_4xN_opcv = T_lcs_to_opcv @ points3d_4xN

# Project
points2d_3xN = K @ points3d_4xN_opcv
points2d_3xN = points2d_3xN / points2d_3xN[2,:]

Let's draw an image

In [ ]:
img = 255 * np.ones((img_height_px, img_width_px, 3), np.uint8)

In [ ]:
rows, cols = points2d_3xN.shape
for i in range(0, cols):    
    if np.isnan(points2d_3xN[0, i]) or np.isnan(points2d_3xN[1, i]):
        continue
    cv.circle(img, (utils.round(points2d_3xN[0, i]), utils.round(points2d_3xN[1, i])), 2, (255, 0, 0), -1)

ret =cv.rectangle(img, (0, 0), (img_width_px, img_height_px), (0,0,0), 2)

In [ ]:
Image.fromarray(img)

Let's see the setup in 3D.

In [ ]:
# Visualize
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

# Let's plot the Vehicle coordinate system
P_lcs_wrt_lcs = np.array([[1.0, 0.0, 0.0, 0.0],[0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]]) 
plot_cs(P_lcs_wrt_lcs, 'lcs', L=2)
plot_cs(P_opcv_wrt_lcs, 'opcv', L=1)

# Plot the LCS points
ax.scatter(xm, ym, zm, c='r')

ax.set_xlim(0, 15)
ax.set_ylim(-6, 6)
ax.set_zlim(0, 6)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')